In [ ]:
!pip install chromadb

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from Product import products

In [ ]:
documents = [p["description"] for p in products]
ids = [p["id"] for p in products]
metadatas = [{"price": p["price"], "category": p["category"]} for p in products]

In [ ]:
documents

['A heavy-duty gaming laptop with RTX graphics and high refresh rate display.',
 'An ultra-light business laptop with long battery life and excellent portability.',
 'A budget-friendly tablet suitable for reading, media consumption, and casual apps.',
 'A workstation laptop designed for data science, machine learning, and 3D rendering.',
 'A compact tablet with stylus support for drawing, note-taking, and creativity.',
 'A foldable smartphone with dual displays optimized for multitasking and productivity.',
 'A rugged outdoor tablet designed for harsh environments with water and shock resistance.',
 'A mid-range smartphone offering excellent camera performance and fast charging.',
 'A smart home speaker with advanced voice assistant features and multi-room audio support.',
 'A premium ultrawide monitor ideal for trading, productivity, and immersive gaming.',
 'A high-performance desktop tower built for 4K video editing and professional workloads.',
 'A wireless noise-cancelling headpho

In [ ]:
metadatas

[{'price': 2400, 'category': 'laptop'},
 {'price': 1300, 'category': 'laptop'},
 {'price': 280, 'category': 'tablet'},
 {'price': 3200, 'category': 'laptop'},
 {'price': 500, 'category': 'tablet'},
 {'price': 1800, 'category': 'smartphone'},
 {'price': 650, 'category': 'tablet'},
 {'price': 550, 'category': 'smartphone'},
 {'price': 220, 'category': 'smart_home'},
 {'price': 900, 'category': 'monitor'},
 {'price': 2800, 'category': 'desktop'},
 {'price': 350, 'category': 'audio'},
 {'price': 300, 'category': 'wearable'},
 {'price': 750, 'category': 'desktop'},
 {'price': 180, 'category': 'accessory'}]

In [ ]:
embeddings = model.encode(documents)

In [ ]:
print("Embedding matrix shape:", embeddings.shape)

Embedding matrix shape: (15, 384)


In [ ]:
client = chromadb.Client()
collection = client.create_collection(name="Samsung_Lab_Products_Collection")

In [ ]:
collection.add(
    embeddings=embeddings.tolist(),
    documents=documents,
    metadatas=metadatas,
    ids=ids
)


In [ ]:
user_query = "I need a laptop good for machine learning and heavy computation."

In [ ]:

query_embedding = model.encode([user_query]).tolist()
query_embedding

[[-0.05465221032500267,
  -0.05787934362888336,
  0.02702108956873417,
  0.005948247853666544,
  0.05387992411851883,
  -0.04018676280975342,
  -0.06453599035739899,
  0.027641385793685913,
  0.020535357296466827,
  0.049344949424266815,
  -0.01037597469985485,
  0.016333531588315964,
  -0.04243306443095207,
  -0.019650040194392204,
  0.04081089794635773,
  -0.01081877015531063,
  0.04572827368974686,
  -0.04267338290810585,
  0.041752081364393234,
  -0.14022748172283173,
  0.008580101653933525,
  0.009224679321050644,
  0.004132366273552179,
  -0.02920888178050518,
  0.08661023527383804,
  -0.02521762251853943,
  0.02369500882923603,
  -0.034788068383932114,
  -0.03759017959237099,
  -0.020149247720837593,
  -0.01286377478390932,
  0.0010740729048848152,
  -0.08604457974433899,
  0.008254999294877052,
  -0.012255949899554253,
  0.011148283258080482,
  0.043753769248723984,
  -0.017808934673666954,
  0.06923754513263702,
  -0.05182681232690811,
  -0.07390045374631882,
  0.0014978159451

In [ ]:
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

In [ ]:
for doc, meta, id in zip(results["documents"][0], results["metadatas"][0], results["ids"][0]):
    print(f"{id}: {doc} | {meta}")

p4: A workstation laptop designed for data science, machine learning, and 3D rendering. | {'category': 'laptop', 'price': 3200}
p2: An ultra-light business laptop with long battery life and excellent portability. | {'category': 'laptop', 'price': 1300}
p1: A heavy-duty gaming laptop with RTX graphics and high refresh rate display. | {'price': 2400, 'category': 'laptop'}


In [ ]:
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
    where={
        "$and": [
            {"category": "laptop"},
            {"price": {"$lt": 3000}}
        ]
    }
)


In [ ]:

for doc, meta, id in zip(results["documents"][0], results["metadatas"][0], results["ids"][0]):
    print(f"{id}: {doc} | {meta}")

p2: An ultra-light business laptop with long battery life and excellent portability. | {'category': 'laptop', 'price': 1300}
p1: A heavy-duty gaming laptop with RTX graphics and high refresh rate display. | {'category': 'laptop', 'price': 2400}
